In [1]:
# # Mount the Google drive onto Colab Virtual Environment
# from google.colab import drive
# drive.mount('/content/drive')
root = '.'
# root = '/content/drive/My Drive/English Dataset'

In [2]:
import pandas as pd
import numpy as np
import string
import os
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, Embedding, Dense, CuDNNLSTM, LSTM
from keras.models import Model
from keras.callbacks import ModelCheckpoint
import tensorflow as tf
import pickle
from datetime import datetime

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
with open(os.path.join(root, 'en-hi_cleaned.en')) as f:
    lines = f.readlines()
    
lines = pd.DataFrame(lines, columns=['Src'])
lines['Tar'] = None

with open(os.path.join(root, 'en-hi_cleaned.hi')) as f:
    lines['Tar'] = f.readlines()

In [4]:
lines.Src = lines.Src.apply(lambda x: x.strip())
lines.Tar = lines.Tar.apply(lambda x: x.strip())
lines.Tar = '<START> ' + lines.Tar + ' <END>'

In [5]:
lines.shape

(70000, 2)

In [6]:
lines = lines[:30000]

In [7]:
lines.sample(10)

,Src,Tar
39,version,<START> संस्करण <END>
69,name x y,<START> नाम <END>
45,widget,<START> विडजेट <END>
50,co mponent,<START> घटक <END>
24,selected accessible,<START> चुने गए एक्सेसेबेल <END>
10,the color and opacity of the highlight fill,<START> हाइलाइट किया गया भराई का रंग और पारदर्...
7,highlight border color,<START> सीमांत बोर्डर के रंग को हाइलाइट करें <...
21,c lear selection,<START> चुनाव को हटाएं <END>
76,offset,<START> ओफसेट <END>
88,uri,<START> यूआरआई <END>


In [8]:
with open(os.path.join(root, 'embeddings.en'), 'rb') as f:
    src_summary = pickle.load(f)
    
with open(os.path.join(root, 'embeddings.hi'), 'rb') as f:
    tar_summary = pickle.load(f)

In [9]:
for word in ['<START>', '<END>']:
    length_of_dict = len(tar_summary['dictionary'])
    tar_summary['dictionary'][word] = length_of_dict
    tar_summary['reverse_dictionary'][length_of_dict] = word
    tar_summary['embeddings'] = np.vstack((tar_summary['embeddings'], np.zeros((tar_summary['embeddings'].shape[1]))))

In [10]:
all_src_words = src_summary['dictionary'].keys()
all_tar_words = tar_summary['dictionary'].keys()

In [11]:
max_len_src = max([len(sentence.split(' ')) for sentence in lines.Src])
max_len_src

10

In [12]:
max_len_tar = max([len(sentence.split(' ')) for sentence in lines.Tar])
max_len_tar

17

In [13]:
num_encoder_tokens = len(all_src_words)
num_decoder_tokens = len(all_tar_words)
(num_encoder_tokens, num_decoder_tokens)

(8333, 4605)

In [14]:
lines = shuffle(lines)
lines.head(10)

,Src,Tar
90,end,<START> अंत <END>
54,hyperlink,<START> हाइपरकड़ी <END>
36,acti on,<START> कार्रवाई <END>
4,a list of plugins that are disabled by default,<START> उन प्लग इनों की सूची जिन्हें डिफोल्ट र...
70,header,<START> शीर्षकः <END>
22,everything,<START> सभी <END>
11,api browser,<START> एपीआई विचरक <END>
41,col lection,<START> संग्रह <END>
86,not implemented,<START> क्रियान्वित नहीं हुआ है <END>
39,version,<START> संस्करण <END>


In [15]:
def encode_source(batch_input, batch_size):
    encoder_input_data = np.zeros((batch_size, max_len_src), dtype='float32')
    
    for i, input_text in enumerate(batch_input):
        for t, word in enumerate(input_text.split(' ')):
            if word not in src_summary['dictionary'].keys():
                word = 'UNK'
                
            encoder_input_data[i, t] = src_summary['dictionary'][word]
            
    return encoder_input_data

def encode_target(batch_output, batch_size):
    decoder_input_data = np.zeros((batch_size, max_len_tar), dtype='float32')
    decoder_target_data = np.zeros((batch_size, max_len_tar, num_decoder_tokens), dtype='float32')
    
    for i, target_text in enumerate(batch_output):
        for t, word in enumerate(target_text.split(' ')):
            if word not in tar_summary['dictionary'].keys():
                word = 'UNK'
                
            if t < len(target_text.split())-1:
                decoder_input_data[i, t-1] = tar_summary['dictionary'][word]
                
            if t > 0:
                decoder_target_data[i, t-1, tar_summary['dictionary'][word]] = 1
                
    return decoder_input_data, decoder_target_data

In [16]:
def generate_batch(X, y, batch_size):
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = encode_source(X[j:j+batch_size], batch_size)
            decoder_input_data, decoder_target_data = encode_target(y[j:j+batch_size], batch_size)
            
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

In [17]:
X, y = lines.Src, lines.Tar
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
X_train.shape, X_test.shape

((90,), (10,))

In [18]:
latent_dim = src_summary['embeddings'].shape[1]

In [19]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 10
epochs = 50

In [20]:
encoder_inputs = Input(shape=(None, ), name='Inputs')
encoder_embeddings = Embedding(num_encoder_tokens, latent_dim, mask_zero=True, weights=[src_summary['embeddings']], name='English_Embeddings')(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True, name='Encoder_LSTM')
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embeddings)
encoder_states = [state_h, state_c]

In [21]:
decoder_inputs = Input(shape=(None, ), name='Outputs')
decoder_embedding_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero=True, weights=[tar_summary['embeddings']], name='Hindi_Embeddings')
decoder_embeddings = decoder_embedding_layer(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True, name='Decoder_LSTM')
decoder_outputs, _, _ = decoder_lstm(decoder_embeddings, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax', name='Dense_Eng_Hin' + str(datetime.today().date()))
decoder_outputs = decoder_dense(decoder_outputs)

In [22]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [23]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

In [24]:
filepath = os.path.join(root, "best_model_en_hi.hdf5")
checkpoint = ModelCheckpoint(filepath, monitor='acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [25]:
model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size, 
                    callbacks=callbacks_list)

Epoch 1/50
9/9 [==============================] - 5s 542ms/step - loss: 6.3163 - acc: 0.0746 - val_loss: 5.2899 - val_acc: 0.0741

Epoch 00001: acc improved from -inf to 0.07461, saving model to ./best_model_en_hi.hdf5


/usr/local/lib/python3.6/site-packages/keras/engine/topology.py:2364: UserWarning: Layer Decoder_LSTM was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'Encoder_LSTM/while/Exit_2:0' shape=(?, 128) dtype=float32>, <tf.Tensor 'Encoder_LSTM/while/Exit_3:0' shape=(?, 128) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  str(node.arguments) + '. They will not be included '


Epoch 2/50
9/9 [==============================] - 3s 331ms/step - loss: 5.9277 - acc: 0.0629 - val_loss: 5.1470 - val_acc: 0.0370

Epoch 00002: acc did not improve
Epoch 3/50
9/9 [==============================] - 3s 347ms/step - loss: 4.9442 - acc: 0.0296 - val_loss: 4.9122 - val_acc: 0.0370

Epoch 00003: acc did not improve
Epoch 4/50
9/9 [==============================] - 3s 327ms/step - loss: 4.5311 - acc: 0.0303 - val_loss: 4.7434 - val_acc: 0.0370

Epoch 00004: acc did not improve
Epoch 5/50
9/9 [==============================] - 3s 334ms/step - loss: 4.2912 - acc: 0.0303 - val_loss: 4.6407 - val_acc: 0.0370

Epoch 00005: acc did not improve
Epoch 6/50
9/9 [==============================] - 3s 338ms/step - loss: 4.1318 - acc: 0.0303 - val_loss: 4.5896 - val_acc: 0.0370

Epoch 00006: acc did not improve
Epoch 7/50
9/9 [==============================] - 3s 308ms/step - loss: 4.0216 - acc: 0.0303 - val_loss: 4.5805 - val_acc: 0.0370

Epoch 00007: acc did not improve
Epoch 8/50
9/9 [


Epoch 00044: acc improved from 0.41254 to 0.42343, saving model to ./best_model_en_hi.hdf5
Epoch 45/50
9/9 [==============================] - 3s 328ms/step - loss: 1.6095 - acc: 0.4304 - val_loss: 4.0608 - val_acc: 0.1481

Epoch 00045: acc improved from 0.42343 to 0.43040, saving model to ./best_model_en_hi.hdf5
Epoch 46/50
9/9 [==============================] - 3s 318ms/step - loss: 1.5354 - acc: 0.4445 - val_loss: 4.0278 - val_acc: 0.1481

Epoch 00046: acc improved from 0.43040 to 0.44453, saving model to ./best_model_en_hi.hdf5
Epoch 47/50
9/9 [==============================] - 4s 415ms/step - loss: 1.4718 - acc: 0.4557 - val_loss: 4.0005 - val_acc: 0.1481

Epoch 00047: acc improved from 0.44453 to 0.45565, saving model to ./best_model_en_hi.hdf5
Epoch 48/50
9/9 [==============================] - 4s 397ms/step - loss: 1.4082 - acc: 0.4754 - val_loss: 3.9744 - val_acc: 0.1481

Epoch 00048: acc improved from 0.45565 to 0.47543, saving model to ./best_model_en_hi.hdf5
Epoch 49/50
9/9 

In [26]:
model.save(os.path.join(root, 'translator_en_hi.h5'))

/usr/local/lib/python3.6/site-packages/keras/engine/topology.py:2364: UserWarning: Layer Decoder_LSTM was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'Encoder_LSTM/while/Exit_2:0' shape=(?, 128) dtype=float32>, <tf.Tensor 'Encoder_LSTM/while/Exit_3:0' shape=(?, 128) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  str(node.arguments) + '. They will not be included '


In [27]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim, ))
decoder_state_input_c = Input(shape=(latent_dim, ))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2 = decoder_embedding_layer(decoder_inputs)

decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2
)

In [28]:
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = tar_summary['dictionary']['<START>']
    
    stop_condition = False
    decoded_sentence = ''
    
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = tar_summary['reverse_dictionary'][sampled_token_index]
        decoded_sentence += ' ' + sampled_char
        
        if (sampled_char == '<END>') or (len(decoded_sentence) > 50):
            stop_condition = True
            
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index
        
        states_value = [h, c]
    return decoded_sentence

In [29]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1

In [30]:
for i in range(10):
    k+=1
    (input_seq, actual_output), _ = next(train_gen)
    decoded_sentence = decode_sequence(input_seq)
    print('Input English sentence:', X_train[k:k+1].values[0])
    print('Actual Hindi Translation:', y_train[k:k+1].values[0][7:-5])
    print('Predicted Hindi Translation:', decoded_sentence[:-5])

Input English sentence: plugin with various methods of selecting accessibles quickly
Actual Hindi Translation:  प्लग इन जिसमें हैं एक्सेसेबेलों को तेजी से चुनने के लिए कई विधियां 
Predicted Hindi Translation:  प्लग प्लग प्लग प्लग प्लग को को को को को को को को
Input English sentence: selected cell
Actual Hindi Translation:  चुनी गई कोशिका 
Predicted Hindi Translation:  चुनी गई कोशिका कोशिका कोशिका कोशिका कोशिका कोशि
Input English sentence: accessible
Actual Hindi Translation:  पहुंचनीय 
Predicted Hindi Translation:  चुने स्तंभ स्तंभ स्तंभ स्तंभ स्तंभ स्तंभ स्तंभ 
Input English sentence: too many selectable children
Actual Hindi Translation:  बहुत अधिक चयनीय शिशु हैं 
Predicted Hindi Translation:  बहुत अधिक चयनीय शिशु हैं हैं हैं हैं हैं हैं है
Input English sentence: value
Actual Hindi Translation:  मान 
Predicted Hindi Translation:  मान मान मान मान मान मान मान मान मान मान मान मा
Input English sentence: toolkit
Actual Hindi Translation:  औजार बक्सा 
Predicted Hindi Translation:  औजार औजा